In [ ]:
import mne
from IPython.display import clear_output
import copy

import matplotlib.pyplot as plt

import os
os.chdir('..')
from _parameters import *

In [ ]:
# Plot settings
plt.rcdefaults()

plt.rcParams['font.family'] = 'Helvetica Neue'
plt.rcParams['svg.fonttype'] = 'none'

In [ ]:
# Left versus right

def get_lvsr(mode):

    cond = ['same', 'rvrs']
    l, r = mode

    same_all, rvrs_all = [], []

    for s in subjects:
        
        tfr = {f'same-{l}': None, f'same-{r}': None, f'rvrs-{l}': None, f'rvrs-{r}': None}
    
        for c in cond:
            for m in mode:
                fname = dirs['tfr'] + f'/tfr_multi_enc1_{c}_{m}_s{s}.h5'
                tfr[f'{c}-{m}'] = mne.time_frequency.read_tfrs(fname)[0]

        lc, rc = f'same-{l}', f'same-{r}'
        same = copy.deepcopy(tfr[lc])
        same.data = ((tfr[lc].data - tfr[rc].data) / (tfr[lc].data + tfr[rc].data)) * 100
        same_all.append(same)

        lc, rc = f'rvrs-{l}', f'rvrs-{r}'
        rvrs = copy.deepcopy(tfr[lc])
        rvrs.data = ((tfr[lc].data - tfr[rc].data) / (tfr[lc].data + tfr[rc].data)) * 100
        rvrs_all.append(rvrs)

        clear_output()

    same_avg = mne.grand_average(same_all)
    rvrs_avg = mne.grand_average(rvrs_all)

    return same_avg, rvrs_avg

In [ ]:
vis_same_avg, vis_rvrs_avg = get_lvsr(mode= ['itemL', 'itemR'])
mot_same_avg, mot_rvrs_avg = get_lvsr(mode= ['respL', 'respR'])

In [ ]:
def plot_lvsr_multi(data, freq, vlim, title):
    tmin, tmax = [0, 0.25]
    fig, axes = plt.subplots(1, 14, figsize=(20, 4))

    for i in range(14):
        data.plot_topomap(tmin=tmin, tmax=tmax, 
                          fmin=freq[0], fmax=freq[1], 
                          vmin=vlim[0], vmax=vlim[1], 
                          colorbar=False, axes = axes[i], show=False)
        axes[i].set_title(f'{tmin} - {tmax} s')
        tmin+=0.25; tmax+=0.25

    fig.suptitle(title)
    plt.show()
    
    fname = dirs['plot'] + f'/topo/lvsr-{title}.svg'
    fig.savefig(fname, format = 'svg', dpi = 300, bbox_inches='tight', transparent=True)

plot_lvsr_multi(vis_same_avg, (8,12), (-7,7), 'visual-same')
plot_lvsr_multi(vis_rvrs_avg, (8,12), (-7,7), 'visual-reverse')

plot_lvsr_multi(mot_same_avg, (13,30), (-7,7), 'motor-same')
plot_lvsr_multi(mot_rvrs_avg, (13,30), (-7,7), 'motor-reverse')

In [ ]:
def plot_lvsr(data, time, freq, vlim, title):

    tmin, tmax = time
    fmin, fmax = freq
    vmin, vmax = vlim

    fig, ax = plt.subplots(figsize=(2, 2))
    data.plot_topomap(tmin=tmin, tmax=tmax, 
                      fmin=fmin, fmax=fmax, 
                      vmin=vmin, vmax=vmax, 
                      colorbar=False, axes = ax)

    plt.show()
    fname = dirs['plot'] + f'/topo/lvsr-{title}.svg'
    fig.savefig(fname, format = 'svg', dpi = 300, bbox_inches='tight', transparent=True)

# First encoding
plot_lvsr(vis_same_avg, (0.4, 1), (8, 12), (-7, 7), title = 'vis-same-1')
plot_lvsr(vis_rvrs_avg, (0.4, 1), (8, 12), (-7, 7), title = 'vis-rvrs-1')

# Second encoding
plot_lvsr(vis_same_avg, (1.9, 2.3), (8, 12), (-7, 7), title = 'vis-same-2')
plot_lvsr(vis_rvrs_avg, (1.7, 2.3), (8, 12), (-7, 7), title = 'vis-rvrs-2')

# Motor second encoding
plot_lvsr(mot_same_avg, (1.7, 3.5), (13, 30), (-7, 7), title = 'mot-same-2')
plot_lvsr(mot_rvrs_avg, (1.7, 3.5), (13, 30), (-7, 7), title = 'mot-rvrs-2')
